In [ ]:
import sys
sys.path
sys.path.append("./keras/lib/python3.4")

# Import libraries
import time
from tqdm import tqdm

import copy

import math

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler

import numpy as np
np.random.seed(123)  # for reproducibility
import numpy.ma as ma

import scipy as sp

import pandas as pd

import theano

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Input
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Model

import os
import gc

raster = 200
crime_type = "E05"
date_from = "2013-06-16"
date_to = "2017-03-08"
dist = 5

# insert main dir
main_dir = ""

In [ ]:
# Load data
dir_neigh = "%s/neighbour_arrays_%d_%s_%s_%s_x_%d_s_select.csv" % (main_dir, raster,crime_type, date_from, date_to, dist)
data_neigh = pd.read_csv(dir_neigh)

data_neigh = data_neigh[data_neigh.date > '2013-07-06']

data_neigh.head()

In [ ]:
# Split data
timesteps = 21

train_days_percent = 0.7
validation_days_percent = 0.2

ids_count = np.array((data_neigh.drop_duplicates("id", inplace = False)).iloc[:,0]).shape[0]
rows = data_neigh.shape[0]

total_days = rows/ids_count
total_batches = int(total_days/timesteps)

train_batches = int(total_batches*train_days_percent)
validation_batches = int(total_batches*validation_days_percent)
test_batches = total_batches - train_batches - validation_batches

train_rows = ids_count * train_batches * timesteps
validation_rows = ids_count * validation_batches * timesteps
test_rows = ids_count * test_batches * timesteps
print("Train rows: %d, validation rows: %d,  test_rows: %d, all: %d < %d" % (train_rows, validation_rows, test_rows, (train_rows+validation_rows+test_rows), rows))

data_neigh_train = data_neigh[0:train_rows]
data_neigh_validation = data_neigh[(train_rows):(train_rows+validation_rows)]
data_neigh_test = data_neigh[(train_rows+validation_rows):(train_rows+validation_rows+test_rows)]

data_neigh_train.sort_values(['id','date'], ascending=[1,1], inplace=True)
data_neigh_validation.sort_values(['id','date'], ascending=[1,1], inplace=True)
data_neigh_test.sort_values(['id','date'], ascending=[1,1], inplace=True)

X_train = data_neigh_train.iloc[:,:-5].values
X_validation = data_neigh_validation.iloc[:,:-5].values
X_test = data_neigh_test.iloc[:,:-5].values

y_train = data_neigh_train.iloc[:, -1].values
y_validation = data_neigh_validation.iloc[:,-1].values
y_test = data_neigh_test.iloc[:,-1].values

n_train = y_train.shape[0]
n_validation = y_validation.shape[0]
n_test = y_test.shape[0]


n_train, n_validation, n_test, (n_train + n_validation + n_test), rows, data_neigh_train.date.min(), data_neigh_validation.date.min(), data_neigh_test.date.min()

In [ ]:
max_y = int(data_neigh["crimecount"].max())

normalize_y = False

if normalize_y:
    y_train /= max_y
    y_validation /= max_y
    y_test /= max_y

normalize_x = True

if normalize_x:
    # normalize from [0, max] to [0, 1] so max + 1 clasess
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train = scaler.fit_transform(X_train)
    X_validation = scaler.fit_transform(X_validation)
    X_test = scaler.fit_transform(X_test)

binary = True

if binary:
    np.clip(y_train, 0, 1, out=y_train)
    np.clip(y_validation, 0, 1, out=y_validation)
    np.clip(y_test, 0, 1, out=y_test)
    n_classes = 2
    
    max_y = 1
    data_neigh.crimecount[data_neigh["crimecount"] > 0] = 1
    hist, bin_edges = np.histogram(data_neigh["crimecount"])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show()
    print(hist)
else:
    n_classes = max_y + 1
    hist, bin_edges = np.histogram(data[y])
    plt.bar(bin_edges[:-1], hist, width = 1)
    plt.xlim(min(bin_edges), max(bin_edges))
    plt.show() 
    print(hist)

In [ ]:
# Reshape, normalize and categorize data
image_shape = 2*dist + 1
timesteps = 21

n_train_b = int(n_train/timesteps)
n_validation_b = int(n_validation/timesteps)
n_test_b = int(n_test/timesteps)

X_train = X_train.reshape(n_train_b, timesteps, image_shape, image_shape,1).astype('float32')
X_validation = X_validation.reshape(n_validation_b, timesteps, image_shape, image_shape,1).astype('float32')
X_test = X_test.reshape(n_test_b, timesteps, image_shape, image_shape,1).astype('float32')

np.save("X_train.npy",X_train)
np.save("X_validation.npy",X_validation)
np.save("X_test.npy",X_test)
       
# Categorize depends on number of classes

categorize = True
if categorize:
    y_train = to_categorical(y_train, 2)
    y_validation = to_categorical(y_validation, 2)
    y_test = to_categorical(y_test, 2)

    y_train = y_train.reshape(n_train_b, timesteps,2)
    y_validation = y_validation.reshape(n_validation_b, timesteps,2)
    y_test = y_test.reshape(n_test_b, timesteps,2)

    np.save("y_train_c.npy",y_train)
    np.save("y_validation_c.npy",y_validation)
    np.save("y_test_c.npy",y_test)
else:
    y_train = y_train.reshape(n_train_b, timesteps,1)
    y_validation = y_validation.reshape(n_validation_b, timesteps,1)
    y_test = y_test.reshape(n_test_b, timesteps,1)
    
    np.save("y_train.npy",y_train)
    np.save("y_validation.npy",y_validation)
    np.save("y_test.npy",y_test)
    
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import copy

from numpy.ma import masked_array

import seaborn as sns


startcolor = '#fff2e5'
midcolor = '#f2a285'
endcolor = '#c40d21'    
cmap1 = col.LinearSegmentedColormap.from_list('own1',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'wrong', cmap=cmap1)

startcolor = '#f3ffe5' 
midcolor = '#b4f185'   
endcolor = 'darkgreen'   
cmap2 = col.LinearSegmentedColormap.from_list('own2',[startcolor,midcolor,endcolor])
plt.cm.register_cmap(name = 'correct', cmap=cmap2)

classes=[0,1]

def find_max_mcc_threshold(true_values, predictions, thresholds):
    max_mcc = -1
    max_threshold = thresholds[0]
    max_predictions = predictions[:]
    stop = 0
    for i in range(1,thresholds.shape[0]):
        tmp_threshold = thresholds[i]
        tmp_predictions = copy.deepcopy(predictions)
        tmp_predictions[tmp_predictions >= tmp_threshold] = int(1)
        tmp_predictions[tmp_predictions < 1] = int(0)
        tmp_mcc = matthews_corrcoef(true_values, tmp_predictions)
        if tmp_mcc > max_mcc:
            max_mcc = tmp_mcc
            max_threshold = tmp_threshold
            max_predictions = copy.deepcopy(tmp_predictions)
            stop = 0
        else:
            stop += 1
        if stop == 3000:
            break;
    return max_mcc, max_threshold, max_predictions.astype(int)


    
def print_save_single_roc_threshold(true_values, predictions, title, fig_name):
    fpr, tpr, thresholds = roc_curve(true_values, predictions)
    prediction_auc = auc(fpr, tpr)
    
    mcc_max, threshold, predictions = find_max_mcc_threshold(true_values, predictions, thresholds)
    acc = accuracy_score(true_values, predictions)
    f1 = f1_score(true_values, predictions)
    th_index = thresholds.tolist().index(threshold)

    title_font = {'size':'17', 'color':'black', 'weight':'normal', 'verticalalignment':'bottom'}
    axis_font = {'size':'15'}
    cb_font = {'size':'15', 'horizontalalignment':'left'}
    font_path = "/usr/share/fonts/truetype/msttcorefonts/Arial.ttf"
    font_prop = font_manager.FontProperties(fname=font_path, size=15)
    
    
    fig = plt.figure(figsize=(15,7))
    gs1 = gridspec.GridSpec(1, 2)
    ax_list = [fig.add_subplot(ss) for ss in gs1]
    ax1 =  ax_list[0]
    ax2 =  ax_list[1]
    lw = 2

    ax1.scatter(fpr[th_index], tpr[th_index], s=75, c="red", label='Optimal threshold \n(cutoff = %0.4f)' % (threshold))
    ax1.plot(fpr, tpr, color="black", lw=lw, label='ROC curve (area = %0.4f)' % prediction_auc)
    ax1.plot([0, 1], [0, 1], color='gray', lw=lw, linestyle='--')
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('False Positive Rate', **axis_font)
    ax1.set_ylabel('True Positive Rate', **axis_font)
    ax1.set_title("Roc analysis",  **title_font)
    ax1.legend(loc="lower right", prop= font_prop)
    
    cm = confusion_matrix(true_values, predictions)
    cm_n = cm / cm.sum(axis=1)[:, np.newaxis]
    cm_f = [[1,0],[1,0]]
    
    mask1 = [[0,  1], [1, 0]]
    mask2 = [[1,  0], [0, 1]]
    cm1 = masked_array(cm_n,mask1)
    cm2 = masked_array(cm_n,mask2)
    cm1f = masked_array(cm_f,mask1)
    cm2f = masked_array(cm_f,mask2)
    
    cmap1=plt.cm.get_cmap("correct")
    cmap2=plt.cm.get_cmap("wrong")
    
    p2f = ax2.imshow(cm2f,interpolation='nearest',cmap=cmap2)
    p1f = ax2.imshow(cm1f,interpolation='nearest',cmap=cmap1)
   
    p2 = ax2.imshow(cm2,interpolation='nearest',cmap=cmap2)
    p1 = ax2.imshow(cm1,interpolation='nearest',cmap=cmap1)
    
    cb2 = plt.colorbar(p2,shrink=0.5)
    cb2.set_clim(0, 1)
    cb2.remove()
    
    cb1 = plt.colorbar(p1,shrink=0.5)
    cb1.set_clim(0, 1)
    cb1.remove()
    
    cb2 = plt.colorbar(p2f,shrink=0.5)
    cb2.set_clim(0, cm1.sum())
    cb2.ax.get_xaxis().labelpad = 10
    cb2.ax.set_xlabel('False', **cb_font)
    cb2.ax.tick_params(labelsize=15)

    cb1 = plt.colorbar(p1f,shrink=0.5)
    cb1.set_clim(0, cm1.sum())
    cb1.ax.get_xaxis().labelpad = 10
    cb1.ax.set_xlabel('True', **cb_font)
    cb1.ax.tick_params(labelsize=15)

    ax2.grid(False)

    ax2.set_title("Confusion matrix\n\nACC: %0.3f   F1: %0.3f   MCC: %0.3f" % (acc, f1, mcc_max), **title_font)

    tick_marks = np.arange(len(classes))
    ax2.set_xticks(tick_marks)
    ax2.set_yticks(tick_marks)
    ax2.set_xticklabels(classes)
    ax2.set_yticklabels(classes)
    
    thresh = 0.6

    ax2.text(0, 0, "%d\n(%0.2f)" % (cm[0,0], cm_n[0,0]), horizontalalignment="center", color="white" if cm_n[0, 0] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 1, "%d\n(%0.2f)" % (cm[1,1], cm_n[1,1]), horizontalalignment="center", color="white" if cm_n[1, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(1, 0, "%d\n(%0.2f)" % (cm[0,1], cm_n[0,1]), horizontalalignment="center", color="white" if cm_n[0, 1] > thresh else "black", fontproperties=font_prop)
    ax2.text(0, 1, "%d\n(%0.2f)" % (cm[1,0], cm_n[1,0]), horizontalalignment="center", color="white" if cm_n[1, 0] > thresh else "black", fontproperties=font_prop)

    ax2.set_ylabel('True label', **axis_font)
    ax2.set_xlabel('Predicted label', **axis_font)
    
    for label in (ax2.get_xticklabels() + ax2.get_yticklabels() + ax1.get_xticklabels() + ax1.get_yticklabels()):
        label.set_fontsize(13)
    
    plt.suptitle(title, fontsize=20)   
    
    plt.tight_layout(pad=4, w_pad=2, h_pad=1)
    plt.savefig("%s/output/python/keras/images/%s.png" % (main_dir, fig_name))
    #plt.show()
    return "%0.5f,%0.5f,%0.5f,%0.5f,%0.5f,%d,%d,%d,%d,%0.5f,%0.5f,%0.5f,%0.5f" % (prediction_auc,acc,f1,mcc_max,threshold,cm[0,0],cm[1,1],cm[0,1],cm[1,0], cm_n[0,0],cm_n[1,1],cm_n[0,1],cm_n[1,0])
    

In [ ]:
X_train = np.load("X_train.npy")
X_validation = np.load("X_validation.npy")
X_test = np.load("X_test.npy")

In [ ]:
categorize = False
if categorize:
    y_train = np.load("y_train_c.npy")
    y_validation = np.load("y_validation_c.npy")
    y_test = np.load("y_test_c.npy")
    y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1],2)
else:
    y_train = np.load("y_train.npy")
    y_validation = np.load("y_validation.npy")
    y_test = np.load("y_test.npy")
    y_test = y_test.reshape(y_test.shape[0]*y_test.shape[1])

In [ ]:
epochs = 1
timesteps = 21
batch_size = timesteps

result_file_dir = "%s/output/python/model_results.csv" % main_dir
data_id = "tm_neigh_data"

def train_test_model_save_results_crnn_sf(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%s.h5" % (main_dir, model_id, data_id)
    model.save(save_dir_model)

    prediction = model.predict(X_test)

    prediction = prediction.reshape(prediction.shape[0]*prediction.shape[1],2)
    result_line = print_save_single_roc_threshold(y_test[:,1], prediction[:,1], "", "roc_%s_%s_th" % (data_id, model_id))
    params = str(model.to_json())
    with open(result_file_dir, 'a') as file:
        file.write("%s,%s,%s,%s\n" % (model_id, data_id, result_line, params))
        
def train_test_model_save_results_crnn(model, model_id):
    model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_validation, y_validation))
    
    save_dir_model = "%s/output/python/keras/models/%s_%s.h5" % (main_dir, model_id, data_id)
    model.save(save_dir_model)

    prediction = model.predict(X_test)

    prediction = prediction.reshape(prediction.shape[0]*prediction.shape[1])
    result_line = print_save_single_roc_threshold(y_test, prediction, "", "roc_%s_%s_th" % (data_id, model_id))
    params = str(model.to_json())
    with open(result_file_dir, 'a') as file:
        file.write("%s,%s,%s,%s\n" % (model_id, data_id, result_line, params))

In [ ]:
# Model 1
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
    
model.add(TimeDistributed(Dense(100, name="first_dense")))
        
model.add(LSTM(50, return_sequences=True, name="lstm_layer"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_1')


In [ ]:
# Model 2
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
    
model.add(TimeDistributed(Dense(100, name="first_dense")))
        
model.add(LSTM(50, return_sequences=True, name="lstm_layer"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_2')


In [ ]:
# Model 3
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(100, name="first_dense")))
        
model.add(LSTM(50, return_sequences=True, name="lstm_layer"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_3')

In [ ]:
# Model 4
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(35, name="first_dense")))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_4')
print("Done")

In [ ]:
# Model 5
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(35, name="first_dense")))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_5')
print("Done")

In [ ]:
# Model 6
# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
                
model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_6')
print("Done.")

In [ ]:
# Model 7
# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
                
model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_7')
print("Done.")

In [ ]:
# Model 8
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
                
model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_8')
print("Done.")

In [ ]:
# Model 9
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
                
model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_9')
print("Done.")

In [ ]:
# Model 10
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(35, name="first_dense")))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer"));
model.add(LSTM(100, return_sequences=True, name="lstm_layer2"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_10')
print("Done")

In [ ]:
# Model 11
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(35, name="first_dense")))
        
model.add(LSTM(21, return_sequences=True, name="lstm_layer"));
model.add(LSTM(21, return_sequences=True, name="lstm_layer2"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_11')
print("Done")

In [ ]:
# Model 12
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))
                
                
model.add(TimeDistributed(Dense(35, name="first_dense")))
        
model.add(LSTM(21, return_sequences=True, name="lstm_layer"));
model.add(LSTM(21, return_sequences=True, name="lstm_layer2"));
model.add(LSTM(21, return_sequences=True, name="lstm_layer3"));
model.add(LSTM(21, return_sequences=True, name="lstm_layer4"));

model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_12')
print("Done")

In [ ]:
# Model 8
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
                
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_13')
print("Done.")

In [ ]:
# Model 8
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_14')
print("Done.")

In [ ]:
# Model 15
# number of convolutional filters
n_filters = 8

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))

model.add(TimeDistributed(Dropout(0.15)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_15')
print("Done.")

In [ ]:
# Model 16
# number of convolutional filters
n_filters = 16

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))

model.add(TimeDistributed(Dropout(0.15)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_16')
print("Done.")

In [ ]:
# Model 17
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))                
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))
model.add(TimeDistributed(Dense(100)))

model.add(TimeDistributed(Dropout(0.15)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_17')
print("Done.")

In [ ]:
# Model 18
# number of convolutional filters
n_filters = 32

# convolution filter size
n_conv = 2

# pooling window size
n_pool = 2

model=Sequential();      

model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv), padding='same'), input_shape=X_train.shape[1:]))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(Conv2D(n_filters, (n_conv, n_conv))))
model.add(TimeDistributed(Activation('relu')))
model.add(TimeDistributed(MaxPooling2D(pool_size=(n_pool, n_pool))))
model.add(TimeDistributed(Dropout(0.15)))

model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(512)))        

model.add(TimeDistributed(Dense(100)))
        
model.add(LSTM(100, return_sequences=True, name="lstm_layer1"));

model.add(TimeDistributed(Dropout(0.15)))
         
model.add(TimeDistributed(Dense(1), name="time_distr_dense_one"))
model.add(TimeDistributed(Activation('sigmoid')))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['binary_accuracy'])

print(model.summary())

train_test_model_save_results_crnn(model, 'crnn_18')
print("Done.")